In [1]:


import numpy as np
from sklearn.datasets import load_boston

from citk.layer import Dense, RBFDense
from citk.model import FFN, ANFIS
from citk.functions import ReLU, Linear, Sigmoid, Tanh
from citk.losses import MSE, MAE, Huber
from citk.optimisers import GeneticAlgorithmOptimizer, SGDOptimizer, ConjugateSGDOptimizer
from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import QuantileTransformer, quantile_transform

dataset = load_boston()
target = np.array(dataset.feature_names) == "DIS"
X = dataset.data[:, np.logical_not(target)]
y = dataset.data[:, target].squeeze()
y_trans = quantile_transform(dataset.data[:, target],
                             n_quantiles=300,
                             output_distribution='normal',
                             copy=True).squeeze()

In [3]:
X_train, y_train = X[:300], np.expand_dims(y_trans[:300], -1)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

In [4]:
X_test = sc.fit_transform(X[300:320])
y_test = np.expand_dims(y_trans[300:320], -1)

In [5]:
model = FFN(input_shape = X_train.shape[1], layer_specs=[RBFDense(3), Dense(1, nonlinearity=Linear)], loss=MSE, reg_coef=0.1)
#model.compile(input_size = X_train.shape[1], output_size=1, layer_sizes=[25])

In [6]:
model = ANFIS(input_shape = X_train.shape[1], num_rules=5, loss=Huber, reg_coef=0.01)

In [7]:
trained, history = model.fit(train_sample=(X_train, y_train),
                   validation_sample=(X_test, y_test),
                   optimiser=GeneticAlgorithmOptimizer(55),
                   batch_size=16,
                   iters=100,
                   verbose=True)

best individual - 0.24821905975610153



/home/vsydorskyi/anaconda3/envs/venv/lib/python3.7/site-packages/citk/optimisers.py:136: RuntimeWarning: invalid value encountered in true_divide
  probas /= probas


ValueError: probabilities contain NaN

In [8]:
trained, history_1 = model.fit(train_sample=(X_train, y_train),
                   validation_sample=(X_test, y_test),
                   optimiser=ConjugateSGDOptimizer(batch_size=1, tol=1e-3, eta=0.001),
                   batch_size=1,
                   iters=20,
                   verbose=True)

train score - 0.09672408004525654
validation loss - 0.11841680383054794
train score - 0.09628638987865344
validation loss - 0.11831404505896147
train score - 0.09584989973573856
validation loss - 0.11821212081142624
train score - 0.09541460960564177
validation loss - 0.11811103102752145
train score - 0.09498051947854609
validation loss - 0.11801077564591973
train score - 0.09454762934568553
validation loss - 0.11791135460438418
train score - 0.09411593919934325
validation loss - 0.117812767839766
train score - 0.09368544903284941
validation loss - 0.11771501528800185
train score - 0.09325615884057928
validation loss - 0.11761809688411098
train score - 0.09282806861795105
validation loss - 0.11752201256219252
train score - 0.09240117836142377
validation loss - 0.11742676225542285
train score - 0.09197548806849526
validation loss - 0.11733234589605253
train score - 0.09155099773769981
validation loss - 0.1172387634154035
train score - 0.09112770736860623
validation loss - 0.1171460147438

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
import pandas as pd
#sns.lineplot(x=range(210), y='train_loss', data=pd.DataFrame(history).append(pd.DataFrame(history_1)).reset_index(drop=True))
#sns.lineplot(x=range(210), y='validation_loss',data=pd.DataFrame(history).append(pd.DataFrame(history_1)).reset_index(drop=True))

In [ ]:
pd.DataFrame(history).append(pd.DataFrame(history_1)).reset_index(drop=True)

In [ ]:
sns.lineplot(x='iteration', y='train_loss', data=pd.DataFrame(history_1))
sns.lineplot(x='iteration', y='validation_loss', data=pd.DataFrame(history_1))

In [ ]:
predicted = trained.predict(X_train)

In [ ]:
predicted, y_train

In [ ]:
sns.lineplot(x=np.linspace(0,1, len(predicted)), y=predicted.ravel())
sns.lineplot(x=np.linspace(0,1, len(y_test)), y=y_test.ravel())

In [ ]:
model.layer_specs[0].parser.get(model.parser.get(model.W_vect, 'Fuzzify0'), 'sigma')

In [ ]:
model.parser.__dict__

In [ ]:
model.W_vect

In [ ]:
model.parser.__dict__['idxs_and_shapes'][<layer.Dense at 0x1d0eb71d9a0>]